# Import libraries

In [1]:
import os, glob, csv, sys, re
from os import path
import pandas as pd
from bs4 import BeautifulSoup

# Define directories

In [2]:
number = '13'
userhome = os.path.expanduser('~')
dataset = userhome + r'/mining-forum/data/forumfiles_' + number + '/'
analyze_dir = userhome + r'/mining-forum/analyze/'

# Counting number of files

In [3]:
file_list = glob.iglob(dataset + '*')
filenum = ';'.join(file_list).split(';')
filenum.sort()
print ("Found " + str(len(filenum)) + " files")

Found 44000 files


# Clean the noisy data (remove duplicate files)

In [4]:
file = []
mid_list = []
num = 0
for files in glob.iglob(dataset + "*"):    
    num += 1
    sys.stdout.write('\rExtracting file number: %i' %(num) + ' / %i' %(len(filenum)) + ' --> %s' %(files))
    sys.stdout.flush()

    f = open(files)
    f = f.read()
    
    file_num = re.search('\/(\d+)', files)
    file_num = file_num.group(1)
    
    try:
        soup = BeautifulSoup(f, 'html.parser')
        topic = soup.find('a', attrs={'class':'MsgSubText'})
        for title in topic.stripped_strings:
            title = title

        msid = []
        soup = BeautifulSoup(f, 'html.parser')
        for message_id in soup.find_all('td', attrs={'class':'MsgR1 vt al MsgSubText'}):
            for mid in message_id.find_all('span', attrs={'class':'SmallText'}):
                for msgid in mid.find_all('a'):
                    msid.append(msgid.text)
        msid = msid[0]

        data = [file_num, msid, title]
        username = []
        uname = []
        href = []
        soup = BeautifulSoup(f, 'html.parser')
        for userlink in soup.find_all('td', attrs={'class':'MsgSpacer'}):
            for user in userlink.find_all('td', attrs={'class':'msgud'}):
                for usr in user.find_all('a', href=True):
                    href.append(usr['href'])                                        
                    uname.append(usr.text)
                    if uname[0] == '':
                        uname[0] = 'Eclipse User'
                    username = [uname[0],href[0]]
                    
                    try:
                        aut_id = re.search('(\d+)', username[1])
                        aut_id = aut_id.group()
                    except:
                        aut_id = '0'
                    
                    username.insert(len(username),aut_id)

        if msid not in mid_list:
            mid_list.append(msid)
            clean_file = [file_num, msid, title, username[0], username[1], username[2]]
            file.append(clean_file)
        else:
            pass

    except:
        pass
    
print ("\nFiltering files are finished")

Extracting file number: 44000 / 44000 --> /Users/YusufNugroho/mining-forum/data/forumfiles_13/84924
Filtering files are finished


In [5]:
file.sort()
file

[['44000',
  'message #142915',
  'Profiling Web Application on JBOSS Attaching an Agent',
  'surbhigl',
  'index.php?t=usrinfo&id=65421&',
  '65421'],
 ['44001',
  'message #142917',
  'Two questions - team work',
  'Krzysztof Adamski',
  'index.php?t=usrinfo&id=10370&',
  '10370'],
 ['44002',
  'message #142919',
  'Get ContenPane and EditParts',
  'Oelerink',
  'index.php?t=usrinfo&id=58335&',
  '58335'],
 ['44003',
  'message #142920',
  'How to add an Item to a popmenu?',
  'Eclipse User',
  '/donate',
  '0'],
 ['44004',
  'message #142923',
  'how to change class file template?',
  'Eclipse User',
  '/donate',
  '0'],
 ['44005',
  'message #142925',
  'JSP Custom Tags',
  'Eclipse User',
  '/donate',
  '0'],
 ['44006',
  'message #142929',
  'Problem manipulating java source',
  'Stefan Holzknecht',
  'index.php?t=usrinfo&id=4239&',
  '4239'],
 ['44007',
  'message #142935',
  'How to feed (memory-only) DOM into XMLTableTreeViewer',
  'Eclipse User',
  '/donate',
  '0'],
 ['44008

In [6]:
len(file)

43980

In [7]:
with open (analyze_dir + 'csv_files/clean_files/with_author_id/clean_files_forum_with_author_id_' + number + '.csv', 'w') as csvfile:
    writers = csv.writer(csvfile, delimiter = ",")
    header = ['file_number', 'message_id', 'title', 'author', 'author_link', 'author_id']
    writers.writerow(header)
    for item in file:
        writers.writerow(item)
print ('CSV file has been created')

CSV file has been created


In [8]:
df_thread = pd.read_csv(analyze_dir + 'csv_files/clean_files/with_author_id/clean_files_forum_with_author_id_' + number + '.csv')
df_thread

,file_number,message_id,title,author,author_link,author_id
0,44000,message #142915,Profiling Web Application on JBOSS Attaching a...,surbhigl,index.php?t=usrinfo&id=65421&,65421
1,44001,message #142917,Two questions - team work,Krzysztof Adamski,index.php?t=usrinfo&id=10370&,10370
2,44002,message #142919,Get ContenPane and EditParts,Oelerink,index.php?t=usrinfo&id=58335&,58335
3,44003,message #142920,How to add an Item to a popmenu?,Eclipse User,/donate,0
4,44004,message #142923,how to change class file template?,Eclipse User,/donate,0
5,44005,message #142925,JSP Custom Tags,Eclipse User,/donate,0
6,44006,message #142929,Problem manipulating java source,Stefan Holzknecht,index.php?t=usrinfo&id=4239&,4239
7,44007,message #142935,How to feed (memory-only) DOM into XMLTableTre...,Eclipse User,/donate,0
8,44008,message #142937,HTML like UI,Guillaume Brocker,index.php?t=usrinfo&id=7684&,7684
9,44009,message #142941,adaptable what does it mean in eclipse pugin.xml?,Eclipse User,/donate,0


# Remove threads if the author is Eclipse Webmaster

In [9]:
cols = ['file_number', 'message_id', 'title', 'author', 'author_link', 'author_id']
new_df_thread = df_thread[cols][df_thread[cols]['author_id'] != 9957]
new_df_thread

,file_number,message_id,title,author,author_link,author_id
0,44000,message #142915,Profiling Web Application on JBOSS Attaching a...,surbhigl,index.php?t=usrinfo&id=65421&,65421
1,44001,message #142917,Two questions - team work,Krzysztof Adamski,index.php?t=usrinfo&id=10370&,10370
2,44002,message #142919,Get ContenPane and EditParts,Oelerink,index.php?t=usrinfo&id=58335&,58335
3,44003,message #142920,How to add an Item to a popmenu?,Eclipse User,/donate,0
4,44004,message #142923,how to change class file template?,Eclipse User,/donate,0
5,44005,message #142925,JSP Custom Tags,Eclipse User,/donate,0
6,44006,message #142929,Problem manipulating java source,Stefan Holzknecht,index.php?t=usrinfo&id=4239&,4239
7,44007,message #142935,How to feed (memory-only) DOM into XMLTableTre...,Eclipse User,/donate,0
8,44008,message #142937,HTML like UI,Guillaume Brocker,index.php?t=usrinfo&id=7684&,7684
9,44009,message #142941,adaptable what does it mean in eclipse pugin.xml?,Eclipse User,/donate,0


In [10]:
new_df_thread.to_csv(analyze_dir + 'csv_files/clean_files/with_author_id/cleaned/threads_without_bot_authors_forum_' + number + '.csv', index=False)
print ("CSV file has been created")

CSV file has been created
